<a href="https://colab.research.google.com/github/VineetKiragi/CMPE-257/blob/main/CMPE_257_Question_Answering_Task_Rahul_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Make sure you have a GPU running
!nvidia-smi

Wed Nov 30 02:30:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 20.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-kfceox7u/farm-haystack_c41c69ec0d394192b68bbd27caad44eb
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-kfceox7u/farm-haystack_c41c69ec0d394192b68bbd27caad44eb
  Resolved https://github.com/deepset-ai/haystack.git to commit af06519fc4eb646ba77d97da261a7dd5ec3fc7f3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.4/228.4 kB 19

In [4]:
from haystack.nodes import FARMReader


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:

reader = FARMReader(model_name_or_path="distilbert-base-uncased-distilled-squad", use_gpu=True)
data_dir = "data/"
# data_dir = "PATH/TO_YOUR/TRAIN_DATA"


In [7]:
reader.train(data_dir=data_dir, train_filename="/content/drive/MyDrive/merged_file.json", use_gpu=True, n_epochs=10, save_dir="my_model",)

Train epoch 9/9 (Cur. train loss: 1.1377): 100%|██████████| 53/53 [00:12<00:00,  4.26it/s]


In [9]:
# Saving the model happens automatically at the end of training into the `save_dir` you specified
# However, you could also save a reader manually again via:
reader.save(directory="my_model")

In [10]:
# If you want to load it at a later point, just do:
new_reader = FARMReader(model_name_or_path="my_model")

In [11]:
reader_eval_results = new_reader.eval_on_file("data/", "/content/drive/MyDrive/merged_file.json", device="cuda")

- instead of giving you full control over which labels to use, this method always returns three types of metrics: combined (no suffix), text_answer ('_text_answer' suffix) and no_answer ('_no_answer' suffix) metrics.
- instead of comparing predictions with labels on a string level, this method compares them on a token-ID level. This makes it unable to do any string normalization (e.g. normalize whitespaces) beforehand.
Hence, results might slightly differ from those of `Pipeline.eval()`
.If you are just about starting to evaluate your model consider using `Pipeline.eval()` instead.
Evaluating: 100%|██████████| 11/11 [00:04<00:00,  2.73it/s]


In [12]:
reader_eval_results

{'EM': 65.9047619047619,
 'f1': 87.49970107476389,
 'top_n_accuracy': 97.71428571428571,
 'top_n': 4,
 'EM_text_answer': 64.4,
 'f1_text_answer': 87.0746861285021,
 'top_n_accuracy_text_answer': 97.6,
 'top_n_EM_text_answer': 64.8,
 'top_n_f1_text_answer': 88.94992682723807,
 'Total_text_answer': 500,
 'EM_no_answer': 96.0,
 'f1_no_answer': 96.0,
 'top_n_accuracy_no_answer': 100.0,
 'Total_no_answer': 25}

In [13]:
context = '''Visakhapatnam is a port city and industrial center in the Indian state of Andhra Pradesh, on the Bay of Bengal. It's known for its many beaches, including Ramakrishna Beach, home to a preserved submarine at the Kursura Submarine Museum. Nearby are the elaborate Kali Temple and the Visakha Museum, an old Dutch bungalow housing local maritime and historical exhibits.'''

In [14]:
new_reader.predict_on_texts("How is the weather of the city?",[context])

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 57.28 Batches/s]


{'query': 'How is the weather of the city?',
 'no_ans_gap': 7.27392840385437,
 'answers': [<Answer {'answer': "It's known for its many beaches", 'type': 'extractive', 'score': 0.3662858009338379, 'context': "n the Indian state of Andhra Pradesh, on the Bay of Bengal. It's known for its many beaches, including Ramakrishna Beach, home to a preserved submarin", 'offsets_in_document': [{'start': 112, 'end': 143}], 'offsets_in_context': [{'start': 60, 'end': 91}], 'document_id': '180afd6b388ea3555a775d177f6e691b', 'meta': {}}>]}

###Inference Using Pipeline


In [15]:
from haystack import Pipeline, Document
from haystack.utils import print_answers
# reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")
p = Pipeline()
p.add_node(component=new_reader, name="Reader", inputs=["Query"])
res = p.run(
    query="What is the historic importance of the city?", documents=[Document(content=context)]
)
print_answers(res,details="medium")

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 51.05 Batches/s]


Query: What is the historic importance of the city?
Answers:
[   {   'answer': "It's known for its many beaches, including Ramakrishna "
                  'Beach, home to a preserved submarine at the Kursura '
                  'Submarine Museum',
        'context': "ay of Bengal. It's known for its many beaches, including "
                   'Ramakrishna Beach, home to a preserved submarine at the '
                   'Kursura Submarine Museum. Nearby are ',
        'score': 0.6153061389923096}]
